# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv data and create a dataframe.
cities_df = pd.read_csv('../WeatherPy/data/city_weather.csv')
# Drop the NA that were inserted for cities without weather info
cities_df.dropna(inplace=True)

In [3]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.120278,-134.969167,73.20,84.0,79.0,8.03,PF,1.599973e+09
1,Pangoa,-12.116667,-73.000000,66.34,95.0,63.0,4.16,PE,1.599973e+09
2,Juegang,32.317370,121.185525,84.99,56.0,80.0,6.55,CN,1.599973e+09
4,Kerman,30.283209,57.078790,71.60,17.0,0.0,4.70,IR,1.599973e+09
5,Tura,47.609242,19.602790,79.20,93.0,100.0,2.95,RU,1.599973e+09
...,...,...,...,...,...,...,...,...,...
995,Krasnoselkup,65.700000,82.466667,58.57,75.0,99.0,17.78,RU,1.599973e+09
996,Sezemice,50.066321,15.850651,57.99,100.0,32.0,2.24,CZ,1.599973e+09
997,Elin Pelin,42.666667,23.600000,64.00,82.0,20.0,3.36,BG,1.599973e+09
998,Pinawa,50.150000,-95.883333,46.09,85.0,21.0,4.16,CA,1.599973e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
coordinates = cities_df[['Lat','Lng']]

# store the figure in a variable so we can continue to add layers to the same map
fig = gmaps.figure()


# Store 'Poverty Rate' in a variable called 'poverty_rate'
humidity_data = cities_df['Humidity'].astype(float)

# Create the heatmap layer using locations and poverty_rate
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

weather_criteria_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & 
              (cities_df['Wind Speed'] < 10) & 
             (cities_df['Cloudiness'] == 0)]

print(f"The list of cities matching the criteria contains {len(weather_criteria_df)} cities.")
weather_criteria_df.reset_index(drop=True, inplace=True)
weather_criteria_df

The list of cities matching the criteria contains 38 cities.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kerman,30.283209,57.078790,71.60,17.0,0.0,4.70,IR,1.599973e+09
1,Ardmore,34.174167,-97.143333,71.01,88.0,0.0,3.80,US,1.599972e+09
2,Caravelas,-17.750000,-39.250000,71.44,85.0,0.0,7.85,BR,1.599973e+09
3,Jalu,29.033333,21.550000,77.63,44.0,0.0,7.52,LY,1.599973e+09
4,Avarua,-21.207778,-159.775000,78.80,88.0,0.0,9.17,CK,1.599973e+09
5,Alta Floresta,-9.900000,-55.900000,73.51,34.0,0.0,3.62,BR,1.599973e+09
6,Awbari,26.583333,12.766667,73.96,34.0,0.0,5.95,LY,1.599973e+09
7,Abu Samrah,24.748333,50.840000,77.20,51.0,0.0,7.54,QA,1.599973e+09
8,Iquitos,-3.748056,-73.247222,75.90,79.0,0.0,2.42,PE,1.599973e+09
9,Marsa Matruh,31.350000,27.233333,71.60,88.0,0.0,2.80,EG,1.599973e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_criteria_df

#### Setup the framework for the API call

* We are going to narrow the search for the first 10 cities.

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in weather_criteria_df.iloc[0:10,:].iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f"Searching within {row['City']}, {row['Country']}...")

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)


    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Searching within Kerman, IR...


/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Searching within Ardmore, US...
Searching within Caravelas, BR...
Searching within Jalu, LY...
Searching within Avarua, CK...
Searching within Alta Floresta, BR...
Searching within Awbari, LY...
Searching within Abu Samrah, QA...
Searching within Iquitos, PE...
Searching within Marsa Matruh, EG...


In [8]:
# Drop the rows that contains NA leaving only the 10 hotels in the list
hotel_df.dropna(inplace=True)
hotel_df

/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kerman,30.283209,57.078790,71.60,17.0,0.0,4.70,IR,1.599973e+09,Kerman
1,Ardmore,34.174167,-97.143333,71.01,88.0,0.0,3.80,US,1.599972e+09,Ardmore
2,Caravelas,-17.750000,-39.250000,71.44,85.0,0.0,7.85,BR,1.599973e+09,Caravelas
3,Jalu,29.033333,21.550000,77.63,44.0,0.0,7.52,LY,1.599973e+09,Jalu
4,Avarua,-21.207778,-159.775000,78.80,88.0,0.0,9.17,CK,1.599973e+09,Avarua District
5,Alta Floresta,-9.900000,-55.900000,73.51,34.0,0.0,3.62,BR,1.599973e+09,São Pedro
6,Awbari,26.583333,12.766667,73.96,34.0,0.0,5.95,LY,1.599973e+09,Awbari‎
7,Abu Samrah,24.748333,50.840000,77.20,51.0,0.0,7.54,QA,1.599973e+09,Abu Samra
8,Iquitos,-3.748056,-73.247222,75.90,79.0,0.0,2.42,PE,1.599973e+09,Iquitos
9,Marsa Matruh,31.350000,27.233333,71.60,88.0,0.0,2.80,EG,1.599973e+09,Marsa Matruh


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hover_text=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))